I am thinking of settling down in Ames, Iowa. But I m not sure how much it will cost me to find a house there. 
Lucky for me, there is old data to help me predict the price of the house of my dreams.

The data I am going to use contains sale information of houses in Ames from 2006 till 2010, it contains 1460 observations and a good number of explanatory variables, 81 variable to be precise.

Before applying any machine learning techniques, which is the fun part of the project, there is a long tidying process, this data is raw and someone has to do the cleaning first.

So here is the plan. 

 1. **Target Variable**: A little visualization won't hurt, it is the same variable I want to predict, this is our VIP 

 2. **The other variables**: a quick look to the non VIP features.

 3. **Cleaning step**: We will then delete some values and outliers with the help of the previous step.

 4. **Interaction of features** : The price changes from house to another, I m going to need some bivariate and multivariate study to detect these changes. 




In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

data=pd.read_csv('../input/train.csv')
data.head()
print("the number of columns and observation of the data is {}".format(data.shape)) 


**Target variable**

My target variable is SalePrice, it is a numerical variable whose statistics are: 
 

In [ ]:
data['SalePrice'].describe()

We can already feel that there will be few outliers since 75% of the observations are under 214000, while the maximum one has a Sale price of 755000. 
But lets not assume anything and plot a histogram first.

In [ ]:
sns.distplot(data['SalePrice'])

Let's not what we notice: 

 1. **The histogram is right skewed** 

 2. **There is one peak**

This feature is not normal, but this won't affect our prediction, as we will use regression that doesn't get influenced by normality. Thus, there is not need to do a transformation

Lets see first which features are the most correlated with 'SalePrice'

In [ ]:
data.describe()

Some features have too many missing values 

In [ ]:
#Features with missing values, if a feature has 15% or more missing values, we delete it.
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
total =(data.isnull().sum()).sort_values(ascending=False)
missing_data=pd.concat([percent,total],axis=1,keys=['Percent','Total'])
missing_data.head(20)
#these features need to be dropped
new_data=data.drop(['PoolQC','MiscFeature','Alley',"Fence",'FireplaceQu','LotFrontage'],1)

In [ ]:
new_data=data.drop((missing_data[missing_data['Total']>1]).index,1)
new_data=new_data.drop(new_data.loc[new_data['Electrical'].isnull()].index)
print('the number of missing features is {}'.format(new_data.isnull().sum().max()))


Dealing with missing values isn't the only thing we need to do for our features, we still have to deal with outliers too, which we will after a quick bivariate analysis.

Since there are a lot of features, I only pick the strongest correlated ones with dependent variable. But wait, it doesn't mean that I am ignoring the others, I m still going to use them when I start the prediction.

In [ ]:
import matplotlib.pyplot as plt
corrmat = data.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

I picked the most correlated ones with **SalesPrice** : **GrLivArea** , **OverallQual**, **TotalBmstSF**, **1stFlrSF** , **GarageArea** and **GarageCars.**


In [ ]:
highly_correlated = ['GrLivArea' ,'TotalBsmtSF' , '1stFlrSF' ,'GarageArea']
for feature in highly_correlated:
    new_data.plot.scatter(x=feature,y="SalePrice")
highly_correlated_cat=['OverallQual',"GarageCars"]
for feature in highly_correlated_cat:
    sns.factorplot(x=feature, y='SalePrice',
                   data=new_data, kind="box",
                   size=6
                  );
    

Thanks to these plots, I spotted four outliers, two have high Sale price, while two has a big value of GRLivArea. I m getting rid of them. 


In [ ]:
new_data['SalePrice'].sort_values(ascending=False)[:2]
new_data=new_data.drop(new_data[new_data['SalePrice']==755000].index)
new_data=new_data.drop(new_data[new_data['SalePrice']==745000].index)

new_data['GrLivArea'].sort_values(ascending=False)[:2]
new_data=new_data.drop(new_data[new_data['GrLivArea']==5642].index)
new_data=new_data.drop(new_data[new_data['GrLivArea']==4676].index)

In [ ]:
new_data.plot.scatter(x='GrLivArea',y="SalePrice")

Alright, much better! 

So I have an idea of the dependent variable and the most important features. I deleted some features with too many missing values and got rid of four observations with four outliers.



Almost forgot dummies, a quick solution to turn categorical data into dummies 

In [ ]:
new_data=pd.get_dummies(new_data)

**Prediction time:** 
well the preparing process surprisingly didn't take much time, but we can always go back to the preparation process if we fail to get a good prediction.


In [ ]:
columns =new_data.columns.tolist()
# Filter the columns to remove ones we don't want.
columns = [c for c in columns if c not in ["Id",'SalePrice']]
target='SalePrice'
columns


In [ ]:
from sklearn.cross_validation import train_test_split
train ,test = train_test_split(new_data,test_size=0.25) 

print(train.head())
print(test.head())

In [ ]:
# Import the linearregression model.
from sklearn.linear_model import LinearRegression

# Initialize the model class.
model = LinearRegression()
# Fit the model to the training data.
model.fit(train[columns], train[target])

In [ ]:
# Import the scikit-learn function to compute error.
from sklearn.metrics import mean_squared_error

# Generate our predictions for the test set.
predictions = model.predict(test[columns])

# Compute error between our test predictions and the actual values.
mean_squared_error(predictions, test[target])